## TEST MODELS INITIALISATION

In [1]:
import sys
sys.path.append('../')

import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

import datasets
import models
import importlib
importlib.reload(datasets)
importlib.reload(models)

<module 'models' from '/home/parley/Mixed_RHM/models/../models/__init__.py'>

## IMPORT A HIERARCHICAL DATASET TO TEST THE MODELS

In [2]:
random.seed()
seed_rules = 12345678   # seed of the random hierarchy model
seed_sample = 99999999  # random.randrange(10000000,99999999)
print('sampling seed:', seed_sample)

train_size=32
test_size=0
input_format='onehot'




dataset=datasets.MixedRandomHierarchyModel(
    num_features=v,     # vocabulary size
    num_classes=n,      # number of classes
    fraction_rules=f,     # number of synonymic low-level representations (multiplicity)
    rule_sequence_type=1,
    s_2=2,
    s_3=3,       # size of the low-level representations
    num_layers=L,       # number of levels in the hierarchy
    seed_rules=0,
    seed_sample=1,
    train_size=train_size,
    test_size=0,
    input_format='onehot',
    whitening=1   
    )

x = dataset.features
y = dataset.labels
print('input: tensor of size', x.size())
print('outputs: tensor of size', y.size())

sampling seed: 99999999
input: tensor of size torch.Size([32, 8, 13])
outputs: tensor of size torch.Size([32])


In [4]:
depth = 3
width = 256
rule_sequence_type = 1

model_cnn = models.hCNN_mixed(rule_sequence_type,v,width,n,depth,bias=True)

print(model_cnn)

for i in range(depth-1):
    print(f'{i+1}-th layer weights, size:', model_cnn.hidden[i][0].filter_2.size())
print(f'readout weights, size:', model_cnn.readout.size())

model_y = x
for i in range(depth-1):
    model_y = model_cnn.hidden[i][0](model_y).relu()
    print(f'{i+1}-th hidden rep. size:', model_y.size())
#model_cnn.hidden[0][0](x)
# print(model_y.size())

hCNN_mixed(
  (hidden): Sequential(
    (0): Sequential(
      (0): MyConv1d_mixed_start_3()
      (1): ReLU()
    )
    (1): Sequential(
      (0): MyConv1d_mixed_start_3()
      (1): ReLU()
    )
    (2): MyLinear()
    (3): ReLU()
  )
)
1-th layer weights, size: torch.Size([256, 8, 2])
2-th layer weights, size: torch.Size([256, 256, 2])
readout weights, size: torch.Size([256, 8])
1-th hidden rep. size: torch.Size([32, 256, 5])
2-th hidden rep. size: torch.Size([32, 256, 2])


In [6]:
model_cnn(x).shape

torch.Size([32, 8])